# GPyTorch Regression Tutorial

In [ ]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

Generate data

In [ ]:
# generate cube
xs = torch.linspace(-1, 1, steps=10)
ys = torch.linspace(-1, 1, steps=10)

# suface points
x_face = torch.cat([
  xs,
  torch.ones_like(ys),
  xs, 
  -torch.ones_like(ys)
])
x_face_noise = x_face + torch.randn_like(x_face) * 0.05
y_face = torch.cat([
  torch.ones_like(ys),
  ys,
  -torch.ones_like(ys),
  ys
])
y_face_noise = y_face + torch.randn_like(x_face) * 0.05

# outer points
x_out = x_face * 1.5
y_out = y_face * 1.5

# inner points
x_in = torch.zeros(1)
y_in = torch.zeros(1)

# training data
x_train = torch.cat([
  x_face_noise, x_out, x_in
], dim=-1)
y_train = torch.cat([
  y_face_noise, y_out, y_in
], dim=-1)
pos_train = torch.stack([x_train, y_train], dim=-1)
z_train = torch.cat([
  torch.zeros_like(x_face_noise),
  torch.ones_like(x_out),
  -torch.ones_like(x_in)
], dim=-1)

In [ ]:
plt.scatter(x_in.numpy(), y_in.numpy(), c='r', marker='x')
plt.scatter(x_out.numpy(), y_out.numpy(), c='b', marker='x')
plt.scatter(x_face_noise.numpy(), y_face_noise.numpy(), c='g', marker='x')

Likelihood

In [ ]:
class ThinPlateRegularizer(gpytorch.kernels.Kernel):
  is_stationary = True

  def __init__(self, dist_prior=None, dist_constraint=None, **kwargs):
    super().__init__(**kwargs)
    self.register_parameter(
      name="max_dist",
      parameter=torch.nn.Parameter(torch.zeros(*self.batch_shape, 1, 1)),
    )
    if dist_constraint is None:
      dist_constraint = gpytorch.constraints.GreaterThan(0.20)
    self.register_constraint("max_dist", dist_constraint)
    if dist_prior is not None:
      self.register_prior(
        "dist_prior",
        dist_prior,
        lambda m: m.length,
        lambda m, v: m._set_length(v),
      )

    @property
    def maxdist(self):
      return self.raw_dist_constraint.transform(self.max_dist)

    @maxdist.setter
    def maxdist(self, value):
      return self._set_maxdist(value)

    def _set_maxdist(self, value):
      if not torch.is_tensor(value):
        value = torch.as_tensor(value).to(self.max_dist)
      self.initialize(max_dist=self.raw_dist_constraint.inverse_transform(value))

    def forward(self, x1, x2, **params):
      diff = self.covar_dist(x1, x2, **params)
      diff.where(diff == 0, torch.as_tensor(1e-20))
      noise = 1e-5
      white = noise * torch.eye(diff.shape[0], diff.shape[1])
      tp = (
        2 * torch.pow(diff, 3)
        - 3 * self.max_dist * torch.pow(diff, 2)
        + self.max_dist**3
      )
      return white + tp


In [ ]:
class thinPlateModel(gpytorch.models.ExactGP):
  def __init__(self, train_x, train_y, likelihood):
    super(thinPlateModel, self).__init__(train_x, train_y, likelihood)
    self.mean_module = gpytorch.means.ZeroMean()
    self.covar_module = ThinPlateRegularizer()

  def forward(self, x):
    mean_x = self.mean_module(x)
    covar_x = self.covar_module(x)
    return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


In [ ]:
# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = thinPlateModel(pos_train, z_train, likelihood)

In [ ]:
hypers = {
  'likelihood.noise_covar.noise': torch.tensor(0.05),
  'covar_module.max_dist': torch.tensor(0.5),
}
model_params = model.initialize(**hypers)

train

In [ ]:
training_iter = 30

In [ ]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
  # Zero gradients from previous iteration
  optimizer.zero_grad()
  # Output from model
  output = model(pos_train)
  # Calc loss and backprop gradients
  output.log_prob(z_train)
  loss = -mll(output, z_train)
  loss.backward()
  print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
    i + 1, training_iter, loss.item(),
    model.covar_module.base_kernel.lengthscale.item(),
    model.likelihood.noise.item()
  ))
  optimizer.step()

predict

In [ ]:
test_x = torch.linspace(0, 1, 100)
f_preds = model(test_x)
y_preds = likelihood(model(test_x))

f_mean = f_preds.mean
f_var = f_preds.variance
f_covar = f_preds.covariance_matrix
f_samples = f_preds.sample(sample_shape=torch.Size([1000]))

In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
  test_x = torch.linspace(0, 1, 51)
  observed_pred = likelihood(model(test_x))

plot

In [ ]:
with torch.no_grad():
  # Initialize plot
  f, ax = plt.subplots(1, 1, figsize=(4, 3))

  # Get upper and lower confidence bounds
  lower, upper = observed_pred.confidence_region()
  # Plot training data as black stars
  ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
  # Plot predictive means as blue line
  ax.plot(test_x.numpy(), observed_pred.mean.numpy(), 'b')
  # Shade between the lower and upper confidence bounds
  ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
  ax.set_ylim([-3, 3])
  ax.legend(['Observed Data', 'Mean', 'Confidence'])
